## Extract Organizations
There is a big mismatch between the contacts listed with meters in the old database and the tblContact table, but Joe indicated that it would be better for him to sort it out later. This will just pull the organization information from the old database and save as CSV.

It appears that these contacts are most accurately described as "land owners", so that will be the saved file name.

In [1]:
import pyodbc
import pandas as pd

In [2]:
#Set up PyODBC connection to Access database
conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=C:\Users\clmbn\NMWaterData\PVACDdb\PVACD AMS Jan2023.accdb;'
    )
cnxn = pyodbc.connect(conn_str)

#### Import Contacts
There are two tables: tblContactX and tblContact. ContactX appears to have several rows of what look like test contacts. Will use tblContact.

Fields: sName, sMailing1, sCity, sState, sZip, sPhone, sPhoneMob, sNote

In [3]:
qry = '''
    SELECT sName, sMailing1, sCity, sState, sZip, sPhone, sPhoneMobile, sNote 
    FROM tblContact ORDER BY sName ASC
'''
contact_df = pd.read_sql_query(qry, cnxn)
contact_df.head()

C:\Users\clmbn\AppData\Local\Temp\ipykernel_6820\466642332.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  contact_df = pd.read_sql_query(qry, cnxn)


,sName,sMailing1,sCity,sState,sZip,sPhone,sPhoneMobile,sNote
0,AG SERVICES,1905 OLD DEXTER HIGHWAY,ROSWELL,NM,88203,575-626-4939,None,NEAL POPE AND RAY POPE
1,AIRGAS,1103 EAST SECOND ST,ROSWELL,NM,88201,575-622-1950,None,None
2,"AMERICAN OXYGEN CO., INC.",None,None,None,None,None,None,None
3,"ARMSTRONG & SON, LLC",P.O. BOX 1973,ROSWELL,NM,88202,575-622-1080,None,None
4,ARTESIA COUNTRY CLUB,P.O. BOX 1305,ARTESIA,NM,88211,575-746-2055,None,None


In [4]:
# Rename columns for final database
contact_df.columns = ['organization_name', 'address', 'city', 'state', 'zip', 'phone', 'mobile', 'note']
contact_df.head()

,organization_name,address,city,state,zip,phone,mobile,note
0,AG SERVICES,1905 OLD DEXTER HIGHWAY,ROSWELL,NM,88203,575-626-4939,None,NEAL POPE AND RAY POPE
1,AIRGAS,1103 EAST SECOND ST,ROSWELL,NM,88201,575-622-1950,None,None
2,"AMERICAN OXYGEN CO., INC.",None,None,None,None,None,None,None
3,"ARMSTRONG & SON, LLC",P.O. BOX 1973,ROSWELL,NM,88202,575-622-1080,None,None
4,ARTESIA COUNTRY CLUB,P.O. BOX 1305,ARTESIA,NM,88211,575-746-2055,None,None


In [5]:
# Add on an id column
contact_df['id'] = contact_df.index + 1
contact_df.head()

,organization_name,address,city,state,zip,phone,mobile,note,id
0,AG SERVICES,1905 OLD DEXTER HIGHWAY,ROSWELL,NM,88203,575-626-4939,None,NEAL POPE AND RAY POPE,1
1,AIRGAS,1103 EAST SECOND ST,ROSWELL,NM,88201,575-622-1950,None,None,2
2,"AMERICAN OXYGEN CO., INC.",None,None,None,None,None,None,None,3
3,"ARMSTRONG & SON, LLC",P.O. BOX 1973,ROSWELL,NM,88202,575-622-1080,None,None,4
4,ARTESIA COUNTRY CLUB,P.O. BOX 1305,ARTESIA,NM,88211,575-746-2055,None,None,5


In [6]:
#Export to CSV
contact_df.to_csv('./api/data/landowners.csv', index=False)

In [7]:
cnxn.close()